In [1]:
path = "/scratch/dqwang/daggerR6/"



In [2]:
!ls $path

bbox_bev_train.pkl
bbox_bev_val.pkl
bbox_train.pkl
bbox_val.pkl
bevlist.txt
files.txt
gtatobbox.py
gt_bbox_val.pkl
images
__pycache__
rec_05280030_clouds_15h8m_x-1616y-724tox-1200y105
rec_05280035_clouds_13h24m_x-845y-1747tox-1247y-2429
rec_05280040_clouds_12h3m_x-412y-652tox-756y-237
rec_05280046_clouds_9h57m_x851y-1184tox1305y-1721
rec_05280051_clouds_8h16m_x-383y-1003tox294y-1187
rec_05280054_clouds_15h27m_x1072y-1175tox964y-1179
rec_05280059_clouds_13h11m_x581y-1194tox1113y-1280
rec_05280104_clouds_14h15m_x585y-344tox575y-1227
rec_05280109_clouds_11h45m_x-897y-1954tox-605y-1730
rec_05280112_clouds_19h13m_x-436y-1177tox533y-1212
rec_05280117_clouds_14h13m_x-1354y-736tox-1637y-874
rec_05280120_clouds_14h36m_x451y-498tox790y-320
rec_05280125_clouds_8h45m_x-330y-1229tox-427y-361
rec_05280130_clouds_9h14m_x509y-1229tox539y-613
rec_05280135_clouds_17h38m_x-156y-1228tox743y-1188
rec_05280136_clouds_8h15m_x1040y-1035tox1071y-1379
rec_05280136_clouds_9h57m_x510y-444tox1361y-732
rec_05280141

In [5]:
import os
with open(os.path.join(path, "train_imgs.txt"), "r") as f:
    images = f.readlines()
    images = [i.strip() for i in images]

In [12]:
def read_float(fname):
    with open(fname, "r") as f:
        nums = f.readlines()
        nums = [float(i.strip()) for i in nums]
    return nums

In [14]:
targets = {}
attrs = ["brakes", "angles", "speeds", "steerings", "thottles"]

for a in attrs:
    targets[a] = read_float(os.path.join(path, "train_"+a+".txt"))

In [24]:
import h5py, cv2
import numpy as np

number_images_per_file = 200
number_rewards = 35
output_path = "/data2/yang_cache/aws_data/daggerR6_h5/"
sensor_names = ['CameraMiddle']
image_cut = [0, None]
image_size = [576, 768]


for h5i in range(len(images) // num_images_per_file):
    # create a new h5py file
    name = os.path.join(output_path, "gta_"+str(h5i).zfill(5)+".h5")
    hf = h5py.File(name, "w")
    data_rewards = hf.create_dataset('targets', (number_images_per_file, number_rewards), 'f')
    sensors={}
    dt = h5py.special_dtype(vlen=np.dtype('uint8'))
    for sensor_name in sensor_names:
        sensors[sensor_name] = hf.create_dataset(sensor_name, (number_images_per_file,), dtype=dt)
    print(h5i)
    for iimage in range(num_images_per_file):
        iid = h5i * num_images_per_file + iimage
        # read the image from the disk 
        image_path = os.path.join(path, images[iid])
        this = cv2.imread(image_path)
        this = this[image_cut[0]:image_cut[1], :, :]
        this = cv2.resize(this, (image_size[1], image_size[0]))
        encoded = np.fromstring(cv2.imencode(".jpg", this, [int(cv2.IMWRITE_JPEG_QUALITY), 80])[1], dtype=np.uint8)
        # finish the image processing
        
        # store them
        sensors['CameraMiddle'][iimage] = encoded
        data_rewards[iimage, 0] = targets["steerings"][iid]
        data_rewards[iimage, 1] = targets["thottles"][iid]
        data_rewards[iimage, 2] = targets["brakes"][iid]        
        data_rewards[iimage, 10]= targets["speeds"][iid]
        data_rewards[iimage, 24]= 2 # direction, to be filled in        
    
    hf.close()        

0


/home/yang/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 